<a href="https://colab.research.google.com/github/gretiere545/corpus/blob/main/corpus_maker_pdf_advanced.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Création d'un PDF avec index alphabétique FR->LANGUE et LANGUE->FR

# Imports

In [79]:
#!/usr/bin/env python
# -*- coding: utf8 -*-
!export PYTHONIOENCODING=utf8

!pip install colorama
!pip install fpdf
!pip install --upgrade arabic-reshaper[with-fonttools]
!pip install arabic_reshaper
!pip install pdf2image
!apt-get install poppler-utils 

# https://pyfpdfbook.wordpress.com/2015/03/22/putting-two-adjacent-multicell-blocks/
import os
import pandas as pd
import numpy as np
import json
import fpdf
from fpdf import FPDF
import re
import arabic_reshaper
from arabic_reshaper import ArabicReshaper
'''
from arabic_reshaper import ArabicReshaper
configuration = {
    'delete_harakat': False,
    'support_ligatures': True,
    'RIAL SIGN': True,  # Replace ر ي ا ل with ﷼
}
'''


from colorama import init
init(autoreset=True)
from colorama import Fore, Back, Style
pd.set_option("display.width",1000)
# Ce bout de code pour pouvoir downloader des fichiers sur le drive
from google.colab import drive
drive.mount('/content/drive')
from google.colab import files

# Ce bout de code pour pouvoir downloader des google sheets dans des dataframes
from google.colab import auth
auth.authenticate_user()
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

from pdf2image import convert_from_path, convert_from_bytes
from pdf2image.exceptions import (
    PDFInfoNotInstalledError,
    PDFPageCountError,
    PDFSyntaxError
)

%cd /content/drive/MyDrive/Trad-Union/Corpus/ASAMLA
import config

#reshaper = ArabicReshaper(configuration=configuration)
reshaper = arabic_reshaper.ArabicReshaper(
    arabic_reshaper.config_for_true_type_font(
        '/content/drive/MyDrive/Trad-Union/Corpus/ASAMLA/resources/Adobe Arabic Regular.ttf',
        arabic_reshaper.ENABLE_ALL_LIGATURES
    )
) 



Reading package lists... Done
Building dependency tree       
Reading state information... Done
poppler-utils is already the newest version (0.62.0-2ubuntu2.12).
0 upgraded, 0 newly installed, 0 to remove and 57 not upgraded.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Trad-Union/Corpus/ASAMLA


## Version et mode debug

In [2]:
version = "2.4.10"
debug = True

## Préparation, import des fontes manquantes

In [ ]:
bebas = files.upload()

In [ ]:
!ls /usr/share/fonts/truetype/dejavu

DejaVuSans-Bold.ttf	 DejaVuSansMono.ttf  DejaVuSerif-Bold.ttf
DejaVuSans.cw127.pkl	 DejaVuSans.pkl      DejaVuSerif.pkl
DejaVuSansMono-Bold.ttf  DejaVuSans.ttf      DejaVuSerif.ttf


In [3]:
# !ls /usr/share/fonts/truetype/noto/
# cyrillique
!apt-get update -qq
!apt-get install -y fonts-dejavu-core -qq

# amharique
!apt-get update
!apt-get install fonts-noto




Selecting previously unselected package fonts-dejavu-core.
(Reading database ... 155250 files and directories currently installed.)
Preparing to unpack .../fonts-dejavu-core_2.37-1_all.deb ...
Unpacking fonts-dejavu-core (2.37-1) ...
Setting up fonts-dejavu-core (2.37-1) ...
Processing triggers for fontconfig (2.12.6-0ubuntu2) ...
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:9

## Import du fichier de paramétrage (med_vac.json)

In [80]:
json_file = 'med_vac_synthese.json'
try:
  with open(json_file) as vk_dict:
      vk_lang_dict = json.load(vk_dict)
except BaseException as e:
  print(e)
  uri="null"

In [34]:
debug=True

In [71]:
if debug is True:
  vk=[]
  vk.append(vk_lang_dict[0])
  vk_lang_dict=vk

In [74]:
vk_lang_dict

[{'alphabet': 'اآٱأإبتثجحخدذرزسشصضطظعغفقكلمنهةوؤيئىء',
  'credits': 'Sonia ZARROUK, Wafa TAHRI',
  'font-family': 'DejaVuSans',
  'font-family-bold': 'DejaVuSans-Bold',
  'font-path': '/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf',
  'glossary-subtitle': 'عربي فرنسي',
  'idx': 1,
  'language': 'Arabe',
  'meta': {'credits-fr': '',
   'credits-tr': '',
   'subtitle-fr': '',
   'subtitle-tr': '',
   'title-fr': '',
   'title-tr': ''},
  'text-direction': 'rtl',
  'trigramme': 'ams',
  'unicode_substition': [],
  'update': 'false',
  'uri': '134rEeVux-FvSxPt4EO4OM1M8PnGHa6P1ewtmnsnRfrs'},
 {'alphabet': 'AaBbCcDdEeFfGgHhIiJjKkLlMmNnOoPpQqRrSsTtUuVvWwXxYyZz',
  'credits': 'Khalida BENHEDDER',
  'font-family': 'NotoSerif-Regular',
  'font-family-bold': 'NotoSerif-Bold',
  'font-path': '/usr/share/fonts/truetype/noto/NotoSerif-Regular.ttf',
  'glossary-subtitle': 'English-French',
  'idx': 2,
  'language': 'Anglais',
  'meta': {'credits-fr': '',
   'credits-tr': '',
   'subtitle-fr': '',
 

## Chargement des thèmes graphiques

In [81]:
vk_color_theme=[
{'name':'Abstract vector geometric pattern. Symmetrical layout. Illustration eps 10.','color_1':'128, 191, 162','color_2':'137, 166, 93','color_3':'217, 184, 85','color_4':'217, 170, 85','color_5':'242, 242, 242'},
{'name':'color theme_IMG_2040','color_1':'220, 118, 70','color_2':'243, 161, 75','color_3':'147, 173, 164','color_4':'191, 219, 207','color_5':'234, 223, 201'},               
{'name':'Water textured background. Calm sea ripples','color_1':'3, 140, 140','color_2':'3, 166, 166','color_3':'3, 127, 140','color_4':'217, 170, 85','color_5':'242, 242, 242'},
{'name':'Pink and blue abstract paper background from a curved sheet.','color_1':'217, 119, 173','color_2':'102, 127, 109','color_3':'208, 217, 242','color_4':'121, 150, 132','color_5':'217, 187, 169'},
{'name':'Sort of blue','color_1':'255, 255, 255','color_2':'81, 129, 140','color_3':'47, 89, 115','color_4':'133, 166, 162','color_5':'60, 60, 255'},
{'name':'asamla','color_1':'255, 255, 255','color_2':'166, 3, 33','color_3':'174, 186, 191','color_4':'242, 188, 27','color_5':'242, 140, 15','color_6':'242, 48, 5'},
{'name':'Healthcare background with medical symbols in hexagonal frame','color_1':'255, 255, 255','color_2':'4, 173, 191','color_3':'167, 235, 242','color_4':'4, 191, 191','color_5':' 3, 166, 150','color_6':'4, 191, 157'},
{'name':'healthcare background with medical symbols in hexagonal frame','color_1':'242, 242, 242','color_2':'122, 191, 179','color_3':' 149, 191, 184','color_4':'39, 140, 11','color_5':'88, 166, 144','color_6':'166, 3, 33'}

]

# img = "26363.jpg"
# credits <a href='https://fr.freepik.com/vecteurs/fond'>Fond vecteur créé par starline - fr.freepik.com</a>

In [82]:
vk_color_theme[1]['name']
tuple(map(int, vk_color_theme[1]['color_1'].split(', ')))

(220, 118, 70)

# Ouverture de la base de données

In [83]:
# Ouverture de la Sheet Centrale (corpus_central_base)
sheet_central = 'https://docs.google.com/spreadsheets/d/1L8YB1aXHUJwUE9AE6xyn_xMHalinGR335Q7lntwbu1U'
wb_central = gc.open_by_url(sheet_central)
#ouverture de l'onglet Corpus dans un DF
t_corpus = wb_central.worksheet('med_vac_synthese')
data_t_corpus = t_corpus.get_all_values()
df_corpus = pd.DataFrame(data_t_corpus[1:], columns=data_t_corpus[0])

# Fonctions

## Fonctions DB

In [84]:
def charge_corpus_langue(df_corpus, langue):
  #ouverture de l'onglet Corpus dans un DF
  df = df_corpus[['uid','expression', langue,'index']]
  return df

### Index normal

In [85]:
# cas des index normaux (français -> langue)
def charge_corpus_langue_rev(uri):
  wb = gc.open_by_key(uri)
  #ouverture de l'onglet Corpus dans un DF
  t_corpus = wb.worksheet('med_vac_synthese')
  data_t_corpus = t_corpus.get_all_values()
  df_corpus = pd.DataFrame(data_t_corpus[1:], columns=data_t_corpus[0])
  return df_corpus


### Index inversé

In [86]:
# tableau de corpus indéxé par langue de traduction
df_corpus_trad = []
# itération sur chaque trigramme de langue
for i in vk_lang_dict:
  if i['update']=='false':
    df = charge_corpus_langue_rev(i['uri'])
    df_corpus_trad.append(df)
  

## Classe FPDF

In [87]:
class PDF(FPDF):
  def __init__(self):
    super().__init__()
    self.WIDTH = 210
    self.HEIGHT = 297
    self.format = 'A4'
    self.unit = 'mm'
    self.set_margins(20.0, 20.0, 20.0)
    self.color_theme = vk_color_theme[7]
    self.color_1 = tuple(map(int, self.color_theme['color_1'].split(', ')))
    self.color_2 = tuple(map(int, self.color_theme['color_2'].split(', ')))   
    self.color_3 = tuple(map(int, self.color_theme['color_3'].split(', ')))
    self.color_4 = tuple(map(int, self.color_theme['color_4'].split(', ')))
    self.color_5 = tuple(map(int, self.color_theme['color_5'].split(', ')))    
    self.color_6 = tuple(map(int, self.color_theme['color_6'].split(', ')))     
    #self.logo_1 = './logo_asamla.jpg'
    self.logo_1 = config.root_path + 'resources/logo-asamla-transparent.png'
    self.font_1 = ('DejaVuSans', '', 10)
    self.font_2 = ('NotoSerif-Regular', '', 20)  
    self.font_credits = ('NotoSerif-Regular', '', 12)  
    self.font_3 = ('DejaVuSans', '', 14)      

    self.font_size_normal = 10.0
    self.font_size_index = 32.0
    self.chosen_font_expr = 'NotoSerif-Regular' 
    self.chosen_font_bold = 'NotoSerif-Bold'    
    self.standard_text_color = (0,0,0)
    self.font_cover_1 = ('BebasNeue-Regular', '', 80)  
    self.font_cover_2 = ('NotoSerif-Regular', '', 40)      
    self.title_1 = 'Lexique'
    self.title_2 = 'Médical'
    self.title_3 = 'Vaccination'
    self.cover_img_1 = config.root_path + 'resources/med-vac-lex-sample_nomargin.png'
    self.cover_background_color = self.color_1
    self.cover_text_color = self.color_6
    self.cover_23_background_color = self.color_6    
    self.cover_4_background_color = self.color_2     

    self.cover_23_background_color = self.color_2    
    self.cover_4_background_color = self.color_5      

    self.print_footer = False
    self.print_header = False
    self.min_height_required_ln = 25.0
    self.line_height = 6.5

    self.root_path=config.root_path
    self.ligne = 1
    self.change_alpha_index = False
    self.MAX_LIGNE_COLONNE = 33
    self.ndx = 0


  def header(self):
    if self.print_header is True:
      '''
      self.set_text_color (*self.cover_text_color)
      self.set_font(*self.font_1)
      self.cell(0, 0, self.title_1 + ' ' + self.title_2, 0, 0, 'L')
      self.cell(0, 0, self.book_title, 0, 0, 'R')      
      self.ln(5)      
      #self.cell(0, 0, self.get_first_word_in_page(), 0, 0, 'L')  
      '''  
      new_y = self.get_y() + 5
      self.set_draw_color (*self.cover_text_color)
      self.set_text_color (*self.cover_text_color)      
      self.line(0, new_y, 210, new_y)
      self.line(self.WIDTH/2, self.get_y() + 5, self.WIDTH/2, self.HEIGHT-(self.t_margin*2))

  def footer(self):
    # Go to 1.5 cm from bottom
    self.set_y(-15)
    # Select Arial italic 8
    self.set_font(*self.font_1)
    self.set_text_color (*self.cover_text_color)
    # Print current and total page numbers
    # Do not print footer on first page 
    if self.print_footer is True:
      #self.cell(0, 10, 'Page %s' % self.page_no() + '/{nb}', 0, 0, 'C')    
      self.cell(0, 10, '%s' % self.page_no(), 0, 0, 'C')        

  def print_page(self, images):
    # Generates the report
    self.add_page()  

  def set_langue(self, langue):
    self.local_language = langue

  def set_credits(self, credits):
    self.credits = credits    

  def set_glossary_subtitle (self, glossary_subtitle):
    self.glossary_subtitle = glossary_subtitle    

  def set_language_special_font (self, language_special_font):
    self.language_special_font = language_special_font    

  def set_text_direction (self, text_direction):
    self.text_direction = text_direction

  def set_book_title (self, book_title):
    self.book_title = book_title

  def add_fonts(self):
    self.add_font('BebasNeue-Regular','', '/content/drive/MyDrive/Trad-Union/Corpus/ASAMLA/BebasNeue-Regular.ttf', uni=True)   
    self.add_font('DejaVuSans', '', '/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf', uni=True)
    self.add_font('DejaVuSans-Bold', '', '/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf', uni=True)    
    
    self.add_font('NotoSerif-Regular','','/usr/share/fonts/truetype/noto/NotoSerif-Regular.ttf', uni=True)    
    self.add_font('NotoSerif-Bold','','/usr/share/fonts/truetype/noto/NotoSerif-Bold.ttf', uni=True)  

    self.add_font('arial_geo','','/content/drive/MyDrive/Trad-Union/Corpus/ASAMLA/arial_geo.ttf', uni=True)    
    self.add_font('arial_geo-bold','','/content/drive/MyDrive/Trad-Union/Corpus/ASAMLA/arial_geo-bold.ttf', uni=True)         
    self.add_font('jiret','','/content/drive/MyDrive/Trad-Union/Corpus/ASAMLA/jiret.ttf', uni=True)            
    
    self.add_font('NotoSerifArmenian-Regular','','/usr/share/fonts/truetype/noto/NotoSerifArmenian-Regular.ttf', uni=True)  
    self.add_font('NotoSerifArmenian-Bold','','/usr/share/fonts/truetype/noto/NotoSerifArmenian-Bold.ttf', uni=True)    
    
    self.add_font('NotoSansArabic-Regular','','/usr/share/fonts/truetype/noto/NotoSansArabic-Regular.ttf', uni=True)  
    self.add_font('NotoNaskhArabic-Regular','','/usr/share/fonts/truetype/noto/NotoNaskhArabic-Regular.ttf', uni=True)    
    self.add_font('DejaVuSerif','','/usr/share/fonts/truetype/dejavu/DejaVuSerif.ttf', uni=True)     

    # https://arabicfonts.net/search
    self.add_font('Adobe Arabic Regular','','/content/drive/MyDrive/Trad-Union/Corpus/ASAMLA/resources/Adobe Arabic Regular.ttf', uni=True)     
    #self.add_font('Adobe Naskh Medium','','/content/drive/MyDrive/Trad-Union/Corpus/ASAMLA/resources/Adobe Naskh Medium.ttf', uni=True)       

    #self.add_font('MarkaziText-Regular', '', '/content/drive/MyDrive/Trad-Union/Corpus/ASAMLA/resources/MarkaziText-Regular.ttf', uni=True)
  
  def set_ndx (self, value):
    self.ndx = value

  def set_effective_page_width(self, value):
    self.effective_page_width = value

  def set_effective_page_height(self, value):
    self.effective_page_height = value

  def set_multi_cell_width(self, value):
    self.multi_cell_width = value

  def set_rowh(self, value):
    self.rowh = value

  def set_colonne (self, colonne):
    self.colonne = colonne

  def set_ligne (self, ligne):
    self.ligne = ligne

  def first_page(self, langue):
    self.set_font(*self.font_2)
    self.set_text_color (*self.cover_text_color)
    # Calculate width of title and position
    w = self.get_string_width(self.title_1) + 6
    self.set_x((self.WIDTH) / 2)    
    self.set_y((self.HEIGHT) /2.5)
    self.cell(0,0, self.title_1 + ' ' + self.title_2, 0, 0, 'C',0)
    self.ln(10)
    self.cell(0,0, self.book_title, 0, 0, 'C',0)
    # auteur traduction
    self.ln(40)
    self.set_font(*self.font_1)
    self.cell(0,0, 'Traduction : ' + self.credits, 0, 0, 'C')

    self.add_page()
    self.set_text_color (0,0,0)

  def second_page(self):
    self.set_font(*self.font_2)
    self.set_text_color (*self.cover_text_color)
    # Calculate width of title and position
    w = self.get_string_width(self.title_1) + 6
    self.set_x((self.WIDTH) / 2)    
    self.set_y((self.HEIGHT) /2.5)
    self.cell(0,0, self.title_3, 0, 0, 'C')

    self.add_page()
    self.set_text_color (0,0,0)    

  def blank_page(self):
    self.add_page()


  def cover_green(self):

    line_feed_height=25 #19
    self.set_fill_color(*self.cover_background_color)
    self.rect(0, 0, self.WIDTH, self.HEIGHT, style = 'F')  
    self.image(self.cover_img_1,0,0,self.WIDTH,self.HEIGHT)  

    self.set_text_color(*self.color_1)
    self.set_xy(self.l_margin, (self.HEIGHT/2))
    self.set_font(*self.font_cover_1)
    self.cell(0,0, self.title_1, 0, 0, 'C')
    self.ln(line_feed_height)  

    self.set_font(*self.font_cover_1)
    self.set_text_color (*self.color_1)
    self.cell(0,0, self.title_2, 0, 0, 'C')    
    self.ln(line_feed_height)

    self.set_font(*self.font_cover_2)
    self.set_text_color (*self.color_1)
    self.cell(0,0, self.book_title, 0, 0, 'C')    
    self.ln(28)

    self.midpage = self.get_y()-14
    self.set_fill_color(*self.cover_background_color)
    #self.rect(0, self.midpage, self.WIDTH/1.5, 30, style = 'F')

    self.set_font(*self.font_credits)
    self.set_text_color (*self.color_1) 
    self.cell(0,0, 'Traduction : ' + self.credits, 0, 0, 'C')

    self.ln(10)
    self.set_font(self.language_special_font,'', 20.0)

    if self.text_direction == 'rtl':
      self.glossary_subtitle = rtl_arabic(self.glossary_subtitle, self.ndx, self)
    # à étudier dans le JSON : glossary_subtitle : ['val1', 'val2']
    #self.cell(0,0, self.glossary_subtitle, 0, 0, 'L')      
    self.set_font(*self.font_2)

    # 0.264583 = constante px -> mm; 0.3 rapport taille image
    t_logo_pixel_sz = (511,135)
    t_logo_mm_sz_reduced = tuple(np.multiply (t_logo_pixel_sz, 0.3))

    self.image(self.logo_1,(self.WIDTH/2)-((t_logo_mm_sz_reduced[0]*0.264583)/2),self.HEIGHT-(self.l_margin*2), t_logo_mm_sz_reduced[1])

    self.add_page()  


  def cover_2(self):
    self.set_fill_color(*self.cover_23_background_color)
    self.rect(0, 0, self.WIDTH, self.HEIGHT, style = 'F')


    # crédits publication
    self.set_font(*self.font_1)
    self.set_text_color (*self.cover_background_color) 
    self.set_xy(self.l_margin,self.HEIGHT-(self.l_margin*2))
    credits = "Adaptation graphique : Gilles Retière, Hammer & Marteau."    
    self.cell(0,0, credits, 0, 0, 'L')
    self.ln(5)  
    credits = "Image de couverture: Freepik.com."
    self.cell(0,0, credits, 0, 0, 'L')
    self.ln(5)      
    credits = "Cette couverture a été conçue en utilisant des ressources de Freepik.com."   
    self.cell(0,0, credits, 0, 0, 'L')     
    self.ln(5)          
    
    self.add_page()  

  def cover_3(self):
    # cette page doit etre impaire (recto) : si elle est paire, on insère une page
    if self.page_no()%2 == 0:
      self.add_page()
    self.set_fill_color(*self.cover_23_background_color)
    self.rect(0, 0, self.WIDTH, self.HEIGHT, style = 'F')
    self.add_page()  

  def cover_4(self):
    #self.image(self.cover_img_1,0,0,self.WIDTH,self.HEIGHT)
    self.set_fill_color(*self.cover_4_background_color)
    #self.rect(0, self.midpage, self.WIDTH, 30, style = 'F')
    self.rect(0, 0, self.WIDTH, self.HEIGHT, style = 'F') 
    # coordonneés ASAMLA
    self.set_font(*self.font_1)
    self.set_text_color (*self.cover_background_color) 
    self.set_xy(self.l_margin,self.HEIGHT-(self.l_margin*2))
    self.cell(0,0, 'ASAMLA', 0, 0, 'L')
    self.ln(5)  
    self.cell(0,0, '5 place Anatole France', 0, 0, 'L')
    self.ln(5)  
    self.cell(0,0, '44000 Nantes', 0, 0, 'L')
    self.ln(5)      
    self.cell(0,0, 'www.asamla.fr', 0, 0, 'L')
    self.ln(5)      

## Fonctions PDF

### Conversion du PDF en images pour le flip book

In [88]:
def pdf2img(trigramme, output_pdf):

  images = convert_from_path(output_pdf)
  dossier = item['trigramme']
  if not os.path.exists(dossier):
    os.makedirs(dossier)

  os.chdir(config.root_path + dossier)     
  sous_dossier = "lex-fr-"+item['trigramme']
  if not os.path.exists(sous_dossier):
    os.makedirs(sous_dossier)

  os.chdir(config.root_path + dossier + "/" + sous_dossier) 
  file = "med-vac-" + sous_dossier + "-"
  i=0
  for img in images:
    i+=1
    img.save(file + '_' + str(i) + ".jpg", 'JPEG')

  os.chdir(config.root_path)
  return
  

In [89]:
def pdf2img_rev(trigramme, output_pdf):

  images = convert_from_path(output_pdf)
  dossier = item['trigramme']
  if not os.path.exists(dossier):
    os.makedirs(dossier)
  os.chdir(config.root_path + dossier)
  sous_dossier = "lex-" + item['trigramme'] + "-fr"
  if not os.path.exists(sous_dossier):
    os.makedirs(sous_dossier)

  os.chdir(config.root_path +  dossier + "/" + sous_dossier) 
  file = "med-vac-" + sous_dossier + "-"
  i=0
  for img in images:
    i+=1
    img.save(file + '_' + str(i) + ".jpg", 'JPEG')

  os.chdir(config.root_path)
  return


In [90]:
def print_word_top_of_page (pdf, df, ind, item, rev, ndx, t_word, pos):
  pdf.set_xy(pdf.l_margin, pdf.t_margin)
  pdf.set_text_color (*pdf.cover_text_color)
  if rev is True:
    pdf.set_font(vk_lang_dict[ndx]['font-family-bold'],'', pdf.font_size_normal)
    if vk_lang_dict[ndx]['text-direction'] == 'rtl':
      if pos == 'L':
        pos = 'R'
      else:
        pos = 'L'
      pdf.cell(0, 0, shorten_word_top_of_page(rtl_arabic(t_word[0], ndx, pdf)), 0, 0, pos)
    else:
      pdf.cell(0, 0, shorten_word_top_of_page(t_word[0]), 0, 0, pos)
  else:
    pdf.set_font(pdf.chosen_font_bold,'', pdf.font_size_normal)  
    pdf.cell(0, 0, shorten_word_top_of_page(t_word[1]), 0, 0, pos)


In [91]:
def shorten_word_top_of_page (w):
  max_len = 20
  if len(w) > max_len:
    w = w[:max_len] + "..."
  return w

In [92]:
def changement_page(pdf, df, ind, item, rev, ndx):
  #Impression du premier mot sur page impaire et dernier mot sur page paire
  print_word_top_of_page (pdf, df, ind, item, rev, ndx, (df[item['trigramme']][ind-1], df["expression"][ind-1]), 'R')
  pdf.add_page()
  print_word_top_of_page (pdf, df, ind, item, rev, ndx, (df[item['trigramme']][ind], df["expression"][ind]), 'L')  
  return

In [93]:
def add_label (x,pdf):
  pdf.set_xy(x, pdf.get_y())  
  pdf.set_text_color (*pdf.cover_text_color) 
  pdf.write(pdf.rowh,"*", 'http://www.fpdf.org')
  pdf.set_text_color (*pdf.standard_text_color)   


In [94]:
def get_x0(text_orientation, margin, printable_width, column, rev):
  dict_z = [{'key': 'ltr', 'value':-1}, {'key': 'rtl', 'value':1}]
  w = printable_width
  m = margin
  t_z = tuple(map(lambda x : x['value'], dict_z))
  z = t_z[0]
  if text_orientation.lower()=='rtl' and rev is True:
    z = t_z[1]
  x = ((w/2) + z*(w/2)) - m*z

  return x

In [95]:
def get_x1(text_orientation, margin, printable_width, column, rev):
  dict_z = [{'key': 'ltr', 'value': -1}, {'key': 'rtl', 'value': 1}]
  w = printable_width
  m = margin
  t_z = tuple(map(lambda x : x['value'], dict_z))
  z = t_z[0]
  if text_orientation.lower()=='rtl' and rev is True:
    z = t_z[1]
  x = (get_x0(text_orientation,m,w,column, rev) - z*(w/2))
  return x

In [96]:
def get_word_tuple_order(ndx, langue, df, pdf, ind, rev, text_direction):
  #par défaut
  w = df[langue][ind]
  if text_direction == 'rtl':
    w = rtl_arabic(w, ndx, pdf) 
  t_word = (df['expression'][ind], w)
  if rev is True and text_direction == 'ltr':
    t_word = (w, df['expression'][ind])
  return t_word

In [97]:
def get_word_tuple_fonts(ndx, langue, df, pdf, ind, rev, text_direction):
  #par défaut
  alignment = 'L'
  t_font = (pdf.chosen_font_bold, vk_lang_dict[ndx]['font-family'])
  if text_direction == 'rtl':
    # cas de l'arabe et du persan
    if rev is True:
      t_font = (vk_lang_dict[ndx]['font-family'], vk_lang_dict[ndx]['font-family-bold'])
    alignment = 'R' 
  return t_font

In [98]:
def get_x_offset(pdf, rev, ndx):
  if pdf.colonne == 1:
    if rev is True and vk_lang_dict[ndx]['text-direction']=='rtl':
      x_offset = (pdf.effective_page_width/2) + pdf.l_margin/2  
    else:
      x_offset = 0     
  else:
    if rev is True and vk_lang_dict[ndx]['text-direction']=='rtl':
      x_offset = 0
    else:
      x_offset = (pdf.effective_page_width/2) + pdf.l_margin/2  

  return x_offset

In [99]:
def get_pos_y(pdf, top_y):
  ybefore = pdf.get_y()
  x_offset = 0
  if pdf.colonne == 1:
    if pdf.ligne == 1:
      if pdf.change_alpha_index == True:
        ybefore = pdf.get_y()
      else:
        ybefore = top_y
  if pdf.colonne == 2:
    if pdf.ligne == 1:
      y_top_col_B = top_y
      if pdf.change_alpha_index == True:
        ybefore = pdf.get_y()
      else:
        ybefore = top_y
    
  return ybefore


## Fonctions d'impression

#### print_word_original()

In [100]:
def print_word_original(pdf, font, word, pos_x, pos_y, ln):
  if debug is True:
    print ("Entering print_word_original()" + word)
  # expression A
  pdf.set_font(font,'', pdf.font_size_normal)
  pdf.set_text_color (*pdf.standard_text_color)

  pdf.set_xy(pos_x, pos_y)
  try:
    pdf.multi_cell(pdf.multi_cell_width, pdf.rowh, word, 0, 'L')
  except Exception as e:
    print(e)

  pdf.ln(ln) # saut de ligne
  pdf.set_ligne(pdf.ligne+1)
  pos_x = pos_x +3

  return pos_x

#### print_word_original_rtl()

In [101]:
def print_word_original_rtl(pdf, font, word, pos_x, pos_y, text_direction, ln):
  # expression A
  pdf.set_font(font,'', 12)
  pdf.set_text_color (*pdf.standard_text_color)

  if pdf.colonne == 2:
    pos_x = pdf.l_margin
  
  pdf.set_xy(pos_x, pos_y)
  if text_direction == 'rtl':
    print ("HERE >>>" + font)
    return print_word_translate_arabic_fix (pdf, font, word, pos_x, pos_y, 'R', ln)  

  pdf.multi_cell(pdf.multi_cell_width, pdf.rowh, word, 0, 'R')

  pdf.ln(ln) # saut de ligne
  pdf.set_ligne(pdf.ligne+1)
  pos_x = pos_x +3

  return pos_x

#### print_word_translate()

In [102]:
def print_word_translate (pdf, font, word, pos_x, pos_y, alignment, ln):
  if debug is True:
    print ("Entering print_word_translate()" + word)  
    
  # expression B
  if alignment == 'R':
    return print_word_translate_arabic_fix (pdf, font, word, pos_x, pos_y, alignment, ln)

  else:
    pdf.set_font(font,'', pdf.font_size_normal)
  pdf.set_text_color (*pdf.standard_text_color)  
  pdf.set_xy(pos_x, pdf.get_y())  

  if test_1 is True:
    print ("NOW PRINTING : " + word)

  pdf.multi_cell(pdf.multi_cell_width, pdf.rowh, word, 0, alignment)
  #pdf.cell(pos_x)
  #pdf.cell(0, 0, word, 0, 0, alignment)
  yafter = pdf.get_y() # on marque la position de y après écriture
  pdf.ln(ln*3) # saut de ligne plus fort entre 2 mots
  pdf.set_ligne(pdf.ligne+1)  

  return yafter

#### print_word_translate_rtl()

In [103]:
def print_word_translate_rtl_NULL (pdf, font, word, pos_x, pos_y, alignment, ln):
  # expression B
  pdf.set_font(font,'', pdf.font_size_normal)
  pdf.set_text_color (*pdf.standard_text_color)  
  if pdf.colonne == 2:
    pos_x = pdf.l_margin  

  vk_word = arabic_multi_line_fix (pdf, font, word, pos_x, pos_y, alignment, ln)
  for index, item in enumerate(vk_word):
    if debug is True:
      print("HERE>>>" + item)
    pdf.set_xy(pos_x, pdf.get_y()) 
    pdf.multi_cell(pdf.multi_cell_width, pdf.rowh, item, 0, 'L')
    yafter = pdf.get_y() # on marque la position de y après écriture
    pdf.ln(ln*3) # saut de ligne plus fort entre 2 mots
    pdf.set_ligne(pdf.ligne+1)  

  return yafter

In [104]:
def print_word_translate_rtl (pdf, font, word, pos_x, pos_y, alignment, ln):
  # expression B
  pdf.set_font(font,'', pdf.font_size_normal)
  pdf.set_text_color (*pdf.standard_text_color)  
  if pdf.colonne == 2:
    pos_x = pdf.l_margin  
  pdf.set_xy(pos_x, pdf.get_y())  
  pdf.multi_cell(pdf.multi_cell_width,  pdf.rowh, word, 0, 'L')
  yafter = pdf.get_y() # on marque la position de y après écriture
  pdf.ln(ln*3) # saut de ligne plus fort entre 2 mots
  pdf.set_ligne(pdf.ligne+1)  

  return yafter

In [105]:
def arabicfix (pdf, font, word, pos_x, pos_y, alignment, ln):
  '''
  s = word
  l = len(s)
  w = pdf.multi_cell_width / 1.5
  p = 12
  vk = s.split(" ")
  print (s + "\nlength:"+str(l)+ "\ncell width:"+str(w)+ "\npica:"+str(p))
  z = ""
  vz = []
  vzr = []
  for item in reversed(vk):
    if len(z) < w:
      z = z + item + " "
    else:    
      vz.append(z.strip())
      z =""
      for item in reversed(vz):
        z = z + item + " "      
      vzr.append(z.strip())
      z =""


  vk = z.split(" ")
  vz.append(z.strip())
  z =""
  for item in reversed(vz):
    z = z + item + " "      
  vzr.append(z.strip())
  z =""  
  '''
  return #vzr

#### arabic_multi_line_fix()
Fonction pour corriger le bug de multiline en arabe.
par défaut, multiline() écrit de gauche à droite et cela créé un décalage vertical en cas de plusieurs lignes

In [106]:
def arabic_multi_line_fix (pdf, font, word, pos_x, pos_y, alignment, ln):
  s = word
  l = len(s)
  w = pdf.multi_cell_width / 1.5
  p = 12
  vk = s.split(" ")
  print (s + "\nlength:"+str(l)+ "\ncell width:"+str(w)+ "\npica:"+str(p))
  z = ""
  vz = []
  for item in reversed(vk):
    if len(z) < w:
      z = z + item + " "
    else:    
      # fin de ligne : on écrit le mot item
      vkr = z.split(" ")
      z =""
      for i in reversed(vkr):
        z = z + i + " "      
      vz.append(z.strip())
      print("MOT 1 : " + z)
      z = item + " "

  # fin de mot : on écrit le mot LAST
  vkr = z.split(" ")
  z =""
  for item in reversed(vkr):
    z = z + item + " "    
  vz.append(z.strip())
  print("MOT 2 : " + z)
  z =""
  for item in reversed(vz):
    z = z + item + " "      
  print("MOT 2 (REV) : " + z)
 
  return vz

#### print_word_translate_arabic_fix()

In [107]:
 def print_word_translate_arabic_fix(pdf, font, word, pos_x, pos_y, alignment, ln):
  if debug is True:
    print ("Entering print_word_translate_arabic_fix " + word)
  pdf.set_font(font,'', 12.0)
  pdf.set_text_color (*pdf.standard_text_color)  

  vk_word = arabic_multi_line_fix (pdf, font, word, pos_x, pos_y, alignment, ln)
  for index, item in enumerate(vk_word):
    pdf.set_xy(pos_x, pdf.get_y()) 
    pdf.multi_cell(pdf.multi_cell_width, pdf.rowh, item, 0, alignment)
    yafter = pdf.get_y() # on marque la position de y après écriture
    pdf.ln(ln*3) # saut de ligne plus fort entre 2 mots
    pdf.set_ligne(pdf.ligne+1)  

  return yafter    

In [108]:
def print_metrics_debug (exp, pdf):
  print (Fore.GREEN + "EXP= " + Style.BRIGHT + exp)
  print (Fore.GREEN + "COL= " + Style.BRIGHT + str(pdf.colonne))


#### rtl_arabic()

In [109]:
def rtl_arabic (s, ndx, pdf):
  if debug is True:
    print ("Entering rtl_arabic")
  #https://pypi.org/project/arabic-reshaper/
  url_pattern = re.compile("[a-zA-ZÀ-ÿ0-9\.]+")
  #on inverse d'abord les mots non arabes pour qu'ils ne soient pas à l'envers après reshape !!
  s = re.sub(url_pattern, lambda m:m.group(0)[::-1], s)  

  reshaper = arabic_reshaper.ArabicReshaper(
  arabic_reshaper.config_for_true_type_font(
      vk_lang_dict[ndx]['font-path'],
      arabic_reshaper.ENABLE_ALL_LIGATURES
    )
  ) 

  arabic_string = reshaper.reshape(s)
  arabic_string = arabic_string[::-1]
  w = pdf.get_string_width(arabic_string) + 6
  # problème d'inversion des parenthèses
  arabic_string = arabic_string.replace('(', '§')
  arabic_string = arabic_string.replace(')', '(')    
  arabic_string = arabic_string.replace('§', ')')

  return arabic_string

#### print_word_colonne()

In [110]:
def print_word_colonne(ndx, langue, df, pdf, ind, rev, text_direction):
  # impression d'une ligne expression/traduction dans la bonne fonte
  # on quitte si la traduction est absente
  if len(df[langue][ind].strip()) == 0:
    return

  word_native = df[langue][ind]
  
  t_fonts = (pdf.chosen_font_bold, vk_lang_dict[ndx]['font-family'])

  # Cas nominal : écriture de gauche à droite
  if text_direction == 'ltr':
    alignment = 'L'
  else:
    # cas de l'arabe et du persan
    if rev is True:
      t_fonts = (vk_lang_dict[ndx]['font-family'], vk_lang_dict[ndx]['font-family-bold'])  

    # Inversion des caractères pour la langue arabe (gauche <--- droite)
    word_native = rtl_arabic(word_native, ndx, pdf) 
    alignment = 'R' 

  #tuple qui contient l'expression en fr et la traduction
  t_word = (df['expression'][ind], word_native)

  # https://github.com/gretiere545/corpus/issues/1#issue-1065267052
  if test_1 is True and ind == 1:
    s = t_word[1]
    s = s.replace(u"\u06cd", u"\u06cc")
    t_word = (df['expression'][ind], s)

    s = rtl_arabic (t_word[1], ndx, pdf)
    print (s)
    for a in s:
      print (a + " : " + a.encode("unicode_escape").decode())

  if rev is True and text_direction == 'ltr':
    t_word = (word_native, df['expression'][ind])
    t_fonts = (vk_lang_dict[ndx]['font-family-bold'], pdf.chosen_font_expr)


  ln = 1.0
  y_top_col_A = 20

  ybefore = get_pos_y(pdf, y_top_col_A)
  x_offset = get_x_offset(pdf, rev, ndx)

  pdf.change_alpha_index == False
  pos_x = pdf.get_x()
  pos_y = ybefore

  if debug == True:
    print_metrics_debug(df['expression'][ind], pdf)

  # expression A
  if rev is True and text_direction == 'rtl': 
    pos_x = pdf.l_margin + (pdf.effective_page_width/2) + pdf.l_margin/2
    pos_x = print_word_original_rtl(pdf, t_fonts[1], t_word[1], pos_x, ybefore, text_direction, ln)
  else:
    pos_x = print_word_original(pdf, t_fonts[0], t_word[0], pdf.l_margin + x_offset, ybefore, ln)

  '''  

  pdf.set_font(t_fonts[0],'', pdf.font_size_normal)
  pdf.set_text_color (*pdf.standard_text_color)

  if rev is True and text_direction == 'rtl':  
    pos_x = get_x0(text_direction, pdf.l_margin, pdf.effective_page_width, pdf.colonne, rev)
  else:
    pos_x = pdf.l_margin + x_offset

  
  pdf.set_xy(pos_x, ybefore)
  #pdf.multi_cell(pdf.multi_cell_width, pdf.rowh, df['expression'][ind])
  pdf.multi_cell(pdf.multi_cell_width, pdf.rowh, t_word[0])

  pdf.ln(ln) # saut de ligne
  pdf.set_ligne(pdf.ligne+1)

  if rev is True and text_direction == 'rtl':  
    pos_x = pos_x -3
  else:
    pos_x = pos_x +3
  '''
  yafter = pdf.get_y()
  # expression B
  if rev is True and text_direction == 'rtl':
    pos_x = pdf.l_margin*2 + (pdf.effective_page_width/2)
    yafter = print_word_translate_rtl(pdf, t_fonts[0], t_word[0], pos_x -3, ybefore, alignment, ln) 
  else:
    yafter = print_word_translate(pdf, t_fonts[1], t_word[1], pdf.l_margin + x_offset +3, ybefore, alignment, ln)  

  '''    
  # expression B
  pdf.set_font(t_fonts[1],'', pdf.font_size_normal)
  pdf.set_text_color (*pdf.standard_text_color)  
  pdf.set_xy(pos_x, pdf.get_y())  
  pdf.multi_cell(pdf.multi_cell_width,  pdf.rowh, t_word[1], 0, alignment)
  #pdf.multi_cell(pdf.multi_cell_width, pdf.rowh, rtl_arabic(expression, pdf), 0, 'R')

  #add_label (pdf.get_x() + x_offset + 3, pdf)
  yafter = pdf.get_y() # on marque la position de y après écriture

  pdf.ln(ln*3) # saut de ligne plus fort entre 2 mots
  pdf.set_ligne(pdf.ligne+1)  
  '''
  # on choisi la nouvelle position de y la plus élevée des 2
  if yafter > pdf.get_y() :
    pdf.set_xy(pdf.l_margin, yafter + ln)

In [111]:
def changement_colonne_page(pdf, df, ind, item, rev, ndx):
  if pdf.colonne == 2:
    changement_page(pdf, df, ind, item, rev, ndx)
    pdf.set_colonne(1)
    pdf.set_ligne (1)
  else :
    pdf.set_colonne(2) 
    pdf.set_ligne (1)
  return
  

In [112]:
def chk_changement_colonne_page(pdf, df, ind, item, rev, ndx):
  space_left = pdf.effective_page_height - pdf.get_y()
  # changement de page si place restante insuffisante
  if space_left < 1:
    if debug is True:
      print ("Colonne " + str(pdf.colonne) + " : " + str(pdf.ligne))
    changement_colonne_page(pdf, df, ind, item, rev, ndx)  

  return

### create_pdf_instance()

In [113]:
def create_pdf_instance(ndx, item, df, book_title, rev):
  if debug is True:
    print("Entering create_pdf_instance() " )
  pdf=PDF()
  pdf.alias_nb_pages()
  pdf.set_langue (item['language'])
  pdf.set_credits (vk_lang_dict[ndx]['credits'])
  pdf.set_glossary_subtitle (vk_lang_dict[ndx]['glossary-subtitle'])  
  pdf.set_language_special_font (vk_lang_dict[ndx]['font-family']) 
  pdf.set_text_direction (vk_lang_dict[ndx]['text-direction'])  
  pdf.set_book_title (book_title)
  pdf.set_ndx (ndx)
  
  pdf.add_fonts()
  pdf.format = 'A4'
  pdf.unit = 'mm'
  pdf.set_margins(20.0, 10.0, 20.0)
  A4_height_inches = 11.6929
  effective_page_width = pdf.w - 2*pdf.l_margin
  effective_page_height = pdf.h - 2*pdf.b_margin
  
  multi_cell_width = (effective_page_width/2)-15
  pdf.set_effective_page_width(effective_page_width)
  pdf.set_effective_page_height(effective_page_height)
  pdf.set_multi_cell_width(multi_cell_width)

  rowh = 3.5
  pdf.set_rowh(rowh)
  ln = 5.5

  # Add new page. Without this you cannot create the document.
  pdf.add_page()
  # couverture
  if debug == False:
    pdf.cover_green()
    pdf.cover_2()
    # Remember to always put one of these at least once.
    pdf.set_font('Times','',10.0) 
    pdf.first_page(item['language'])
    pdf.blank_page()
    pdf.second_page()

    pdf.print_header = True  
    pdf.blank_page()
    pdf.print_footer = True  
    pdf.ln(ln)

  idx = '' # index alpha (start)
  # pour chaque élement expression/traduction
  pdf.print_header = True
  pdf.print_footer = True

  if vk_lang_dict[ndx]['text-direction'] == 'rtl' and rev is True:
    pdf.set_colonne(1)
  else:
    pdf.set_colonne(1)

  # Début de l'itération sur le dataframe
  print_word_top_of_page (pdf, df, 0, item, rev, ndx, (df[item['trigramme']][0], df["expression"][0]), 'L')  

  for ind in df.index:
    ybefore = pdf.get_y()
    chk_changement_colonne_page(pdf, df, ind, item, rev, ndx)
    if df['index'][ind] != idx:
      if pdf.ligne > 1:
          pdf.ln(6)
          chk_changement_colonne_page(pdf, df, ind, item, rev, ndx)
          if pdf.ligne > pdf.MAX_LIGNE_COLONNE:
            changement_colonne_page(pdf, df, ind, item, rev, ndx) 

      print_alpha_index(pdf, df, ndx, ind, rev)
      idx = df['index'][ind]
      chk_changement_colonne_page(pdf, df, ind, item, rev, ndx)

    # écriture d'une ligne
    print_word_colonne (ndx, item['trigramme'], df, pdf, ind, rev, vk_lang_dict[ndx]['text-direction'])
    #print_word_tuple(ndx, item['trigramme'], df, pdf, ind, rev, vk_lang_dict[ndx]['text-direction'])

  df_size = len(df.index)-1
  pos='L'
  if pdf.colonne == 2:
    pos = 'R'
  print_word_top_of_page (pdf, df, df_size, item, rev, ndx, (df[item['trigramme']][df_size], df["expression"][df_size]), pos)  
  pdf.ln(ln)  
  pdf.print_header = False
  pdf.add_page()
  pdf.print_footer = False  
  pdf.cover_3()
  pdf.cover_4()
  # cut here -------------------------------------------------------------

  dossier = item['trigramme']
  #if not os.path.exists(dossier):
  #  os.makedirs(dossier)
  #os.chdir("/content/drive/MyDrive/Trad-Union/Corpus/ASAMLA/" + dossier)

  if rev is True:
    output_pdf = 'med-vac-lex-' + item['trigramme'] + '-fr-v' + version + '.pdf'
    pdf.output(output_pdf, 'F')
  else:
    output_pdf = 'med-vac-lex-fr-' + item['trigramme'] + '-v' + version + '.pdf'
    pdf.output(output_pdf, 'F')  

  #os.chdir("/content/drive/MyDrive/Trad-Union/Corpus/ASAMLA/")

  return (output_pdf)

In [114]:
def print_alpha_index(pdf, df, ndx, ind, rev):
  
  ln = 1.0
  y_top_col_A = 20

  ybefore = get_pos_y(pdf, y_top_col_A)

  if ind==0 and rev is True and vk_lang_dict[ndx]['text-direction']=='rtl':
    x_offset = (pdf.effective_page_width/2) + pdf.l_margin/2
  else:
    x_offset = get_x_offset(pdf, rev, ndx)

  alpha_length = len(df.loc[df['index']==df['index'][ind]])
  if debug is True:
    print("Changement d'index alpha! " + str(df['index'][ind]) + ". Qty of words:" + str(alpha_length))
    print("Position : Col="+str(pdf.colonne)+". Ligne=" + str(pdf.ligne))
    print("Position : X="+str(pdf.l_margin + x_offset + 2))    
  pdf.change_alpha_index == False
  pos_x = pdf.get_x()
  pos_y = ybefore
  if rev is True:
    pdf.set_font(vk_lang_dict[ndx]['font-family-bold'],'', pdf.font_size_index)
  else:
    pdf.set_font(pdf.chosen_font_bold,'', pdf.font_size_index)
  pdf.set_text_color (*pdf.cover_text_color)
  
  pdf.set_xy(pdf.l_margin + x_offset + 2, ybefore)      
  if vk_lang_dict[ndx]['text-direction']=='rtl' and rev is True:
    pdf.multi_cell(pdf.multi_cell_width,  pdf.rowh, rtl_arabic(df['index'][ind], ndx, pdf), 0, 'R') 
  else:     
    pdf.multi_cell(pdf.multi_cell_width,  pdf.rowh, df['index'][ind], 0, 'L')
 
  yafter = pdf.get_y() # on marque la position de y après écriture
  pdf.set_ligne(pdf.ligne+1)
  pdf.ln(ln*6) # saut de ligne plus fort entre 2 mots
  # on choisi la nouvelle position de y la plus élevée des 2
  if yafter > pdf.get_y() :
    pdf.set_xy(pdf.l_margin, yafter + ln)


In [115]:
v_ = version.split(".")
v = int(v_[2])+1
version = str(v_[0]+"."+v_[1]+"."+ str(v))

In [116]:
version

'2.4.12'

In [117]:
test_1=False

In [50]:
print(ord(u'ي'))

1610


# Execution du PIPE

In [ ]:
#ouverture de chaque onglet Corpus de travail dans un DF
df = df_corpus_trad.copy()
df_corpus_langue=[] 
os.chdir(config.root_path)
if debug is True:
  print (config.root_path)
for index, item in enumerate(vk_lang_dict):
  if debug is True:
    print(df[index])
  book_title = "Français-"+item['language']
  df_corpus_langue.append (charge_corpus_langue(df_corpus, item['trigramme']))
  output_pdf = create_pdf_instance(index, item, df_corpus_langue[index], book_title, rev=False)
  '''
  pdf2img(item['trigramme'], output_pdf)
  '''
  book_title = item['language'] + "-Français"
  output_pdf = create_pdf_instance(index, item, df[index], book_title, rev=True)
  '''
  pdf2img_rev(item['trigramme'], output_pdf)  
  ''' 


/content/drive/MyDrive/Trad-Union/Corpus/ASAMLA/
          uid  ... index
0    c1cc5bb5  ...     ا
1    ef08cf91  ...     ا
2    b293c557  ...     ا
3    949d9853  ...     ا
4    523bb66a  ...     ا
..        ...  ...   ...
166  e20e9359  ...     ه
167  ab887b0e  ...     و
168  66ef76c1  ...     و
169  3db807c4  ...     و
170  0e4332bf  ...      

[171 rows x 8 columns]
Entering create_pdf_instance() 
Changement d'index alpha! A. Qty of words:25
Position : Col=1. Ligne=1
Position : X=22.0
Entering rtl_arabic
EXP= A jeun
COL= 1
Entering print_word_original()A jeun
Entering print_word_translate()ﻢﺋﺎﺻ
Entering print_word_translate_arabic_fix ﻢﺋﺎﺻ
ﻢﺋﺎﺻ
length:4
cell width:46.66718518518517
pica:12
MOT 2 :  ﻢﺋﺎﺻ 
MOT 2 (REV) : ﻢﺋﺎﺻ 
Entering rtl_arabic
EXP= Accès fébrile concomitant chez un autre membre de la famille ?
COL= 1
Entering print_word_original()Accès fébrile concomitant chez un autre membre de la famille ?
Entering print_word_translate()ةﺮﺳﻷا ﻦﻣ ﺮﺧا دﺮﻓ ﺪﻨﻋ ﺔﻨﻣاﺰﻣ ﻰﻤﺣ ﺔﺑﻮﻧ
Enteri

/usr/local/lib/python3.7/dist-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -65075
  warnings.warn("cmap value too big/small: %s" % cm)
/usr/local/lib/python3.7/dist-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -65076
  warnings.warn("cmap value too big/small: %s" % cm)
/usr/local/lib/python3.7/dist-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -65080
  warnings.warn("cmap value too big/small: %s" % cm)
/usr/local/lib/python3.7/dist-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -65082
  warnings.warn("cmap value too big/small: %s" % cm)
/usr/local/lib/python3.7/dist-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -65083
  warnings.warn("cmap value too big/small: %s" % cm)
/usr/local/lib/python3.7/dist-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -65084
  warnings.warn("cmap value too big/small: %s" % cm)
/usr/local/lib/python3.7/dist-packages/fpdf/tt

Entering create_pdf_instance() 
Entering rtl_arabic
Changement d'index alpha! ا. Qty of words:26
Position : Col=1. Ligne=1
Position : X=117.00077777777776
Entering rtl_arabic
Entering rtl_arabic
EXP= Rougeurs
COL= 1
HERE >>>DejaVuSans-Bold
Entering print_word_translate_arabic_fix راﺮﻤﺣا
راﺮﻤﺣا
length:6
cell width:46.66718518518517
pica:12
MOT 2 :  راﺮﻤﺣا 
MOT 2 (REV) : راﺮﻤﺣا 
Entering rtl_arabic
EXP= Test sérologique (prise de sang)
COL= 1
HERE >>>DejaVuSans-Bold
Entering print_word_translate_arabic_fix ةدﺎﻀﻤﻟا مﺎﺴﺟﻻا رﺎﺒﺘﺧا
ةدﺎﻀﻤﻟا مﺎﺴﺟﻻا رﺎﺒﺘﺧا
length:21
cell width:46.66718518518517
pica:12
MOT 2 :  ةدﺎﻀﻤﻟا مﺎﺴﺟﻻا رﺎﺒﺘﺧا 
MOT 2 (REV) : ةدﺎﻀﻤﻟا مﺎﺴﺟﻻا رﺎﺒﺘﺧا 
Entering rtl_arabic
EXP= Test PCR (dans le nez)
COL= 1
HERE >>>DejaVuSans-Bold
Entering print_word_translate_arabic_fix (ﻒﻧﻻا ﻲﻓ ) را ﻲﺳ ﻲﺑ رﺎﺒﺘﺧا
(ﻒﻧﻻا ﻲﻓ ) را ﻲﺳ ﻲﺑ رﺎﺒﺘﺧا
length:26
cell width:46.66718518518517
pica:12
MOT 2 :  (ﻒﻧﻻا ﻲﻓ ) را ﻲﺳ ﻲﺑ رﺎﺒﺘﺧا 
MOT 2 (REV) : (ﻒﻧﻻا ﻲﻓ ) را ﻲﺳ ﻲﺑ رﺎﺒﺘﺧا 
Entering rtl_arabic
EXP= Test 

/usr/local/lib/python3.7/dist-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -65095
  warnings.warn("cmap value too big/small: %s" % cm)
/usr/local/lib/python3.7/dist-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -65099
  warnings.warn("cmap value too big/small: %s" % cm)
/usr/local/lib/python3.7/dist-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -65101
  warnings.warn("cmap value too big/small: %s" % cm)
/usr/local/lib/python3.7/dist-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -65104
  warnings.warn("cmap value too big/small: %s" % cm)
/usr/local/lib/python3.7/dist-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -65112
  warnings.warn("cmap value too big/small: %s" % cm)
/usr/local/lib/python3.7/dist-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -65120
  warnings.warn("cmap value too big/small: %s" % cm)
/usr/local/lib/python3.7/dist-packages/fpdf/tt

Streaming output truncated to the last 5000 lines.
COL= 2
Entering print_word_original()Digestif
Entering print_word_translate()კუჭის მომნელებელი
EXP= Diphtérie
COL= 2
Entering print_word_original()Diphtérie
Entering print_word_translate()დიფტერია
EXP= Dose
COL= 2
Entering print_word_original()Dose
Entering print_word_translate()დოზა
EXP= Douleur
COL= 2
Entering print_word_original()Douleur
Entering print_word_translate()ტკივილი
Changement d'index alpha! E. Qty of words:10
Position : Col=2. Ligne=20
Position : X=117.00077777777776
EXP= Ecoulement
COL= 2
Entering print_word_original()Ecoulement
Entering print_word_translate()სისველე
EXP= Effet secondaire
COL= 2
Entering print_word_original()Effet secondaire
Entering print_word_translate()გვერდითი მოვლენები
EXP= Enceinte
COL= 2
Entering print_word_original()Enceinte
Entering print_word_translate()ორსულობა
EXP= Epaule
COL= 2
Entering print_word_original()Epaule
Entering print_word_translate()მხარი
EXP= Est-ce que votre enfant va à la crèc

...

Entering create_pdf_instance() 
Entering rtl_arabic
Entering rtl_arabic
EXP= SIDA - Syndrome d'Immuno Déficience Acquise
COL= 1
HERE >>>Adobe Arabic Regular
Entering print_word_translate_arabic_fix ﯼﻮﺷ ﻪﺳﻻﺮﺗ ﻡﻭﺭډﻦﺳ ﺕښﻢﮐ ﺩ ﺖﯿﻓﺎﻌﻣ ﺩ - AIDS
ﯼﻮﺷ ﻪﺳﻻﺮﺗ ﻡﻭﺭډﻦﺳ ﺕښﻢﮐ ﺩ ﺖﯿﻓﺎﻌﻣ ﺩ - AIDS
length:39
cell width:46.66718518518517
pica:12
MOT 2 :  ﯼﻮﺷ ﻪﺳﻻﺮﺗ ﻡﻭﺭډﻦﺳ ﺕښﻢﮐ ﺩ ﺖﯿﻓﺎﻌﻣ ﺩ - AIDS 
MOT 2 (REV) : ﯼﻮﺷ ﻪﺳﻻﺮﺗ ﻡﻭﺭډﻦﺳ ﺕښﻢﮐ ﺩ ﺖﯿﻓﺎﻌﻣ ﺩ - AIDS 
Entering rtl_arabic
EXP= AME - Aide médicale d'État
COL= 1
HERE >>>Adobe Arabic Regular
Entering print_word_translate_arabic_fix ﻪﺘﺳﺮﻣ ﻲﺒﻃ ﺖﻣﻮﮑﺣ ﺩ - AME
ﻪﺘﺳﺮﻣ ﻲﺒﻃ ﺖﻣﻮﮑﺣ ﺩ - AME
length:23
cell width:46.66718518518517
pica:12
MOT 2 :  ﻪﺘﺳﺮﻣ ﻲﺒﻃ ﺖﻣﻮﮑﺣ ﺩ - AME 
MOT 2 (REV) : ﻪﺘﺳﺮﻣ ﻲﺒﻃ ﺖﻣﻮﮑﺣ ﺩ - AME 
Entering rtl_arabic
EXP= BCG - Vaccin bilié de Calmette et Guérin
COL= 1
HERE >>>Adobe Arabic Regular
Entering print_word_translate_arabic_fix ﻦﯿﺴﮐﺍﻭ biliary Guérin ﻭﺍ Calmette - BCG
ﻦﯿﺴﮐﺍﻭ biliary Guérin ﻭﺍ Calmette - BCG
length:39
cell width:46.66718518518517
pica:12
MOT

/usr/local/lib/python3.7/dist-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -64250
  warnings.warn("cmap value too big/small: %s" % cm)
/usr/local/lib/python3.7/dist-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -64283
  warnings.warn("cmap value too big/small: %s" % cm)
/usr/local/lib/python3.7/dist-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -64295
  warnings.warn("cmap value too big/small: %s" % cm)
/usr/local/lib/python3.7/dist-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -64297
  warnings.warn("cmap value too big/small: %s" % cm)
/usr/local/lib/python3.7/dist-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -64379
  warnings.warn("cmap value too big/small: %s" % cm)
/usr/local/lib/python3.7/dist-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -64399
  warnings.warn("cmap value too big/small: %s" % cm)
/usr/local/lib/python3.7/dist-packages/fpdf/tt

          uid  ... index
0    f49563a9  ...      
1    f6e3da7d  ...      
2    36d5b0ee  ...      
3    0e64ab4e  ...      
4    19a0e5a4  ...      
..        ...  ...   ...
166  f397cda1  ...      
167  6d8132e1  ...      
168  58fd3f82  ...      
169  5eb33526  ...      
170  41b26822  ...      

[171 rows x 8 columns]
Entering create_pdf_instance() 
Changement d'index alpha! A. Qty of words:25
Position : Col=1. Ligne=1
Position : X=22.0
Entering rtl_arabic
EXP= A jeun
COL= 1
Entering print_word_original()A jeun
Entering print_word_translate()ﻪﻨﺳﺮﮔ
Entering print_word_translate_arabic_fix ﻪﻨﺳﺮﮔ
ﻪﻨﺳﺮﮔ
length:5
cell width:46.66718518518517
pica:12
MOT 2 :  ﻪﻨﺳﺮﮔ 
MOT 2 (REV) : ﻪﻨﺳﺮﮔ 
Entering rtl_arabic
EXP= Accès fébrile concomitant chez un autre membre de la famille ?
COL= 1
Entering print_word_original()Accès fébrile concomitant chez un autre membre de la famille ?
Entering print_word_translate()؟ﻩﺩﺍﻮﻧﺎﺧ ﯼﺎﻀﻋﺍ ﺯﺍ ﺮﮕﯾﺩ ﯽﮑﯾ ﺭﺩ ﺭﺍﺩ ﺐﺗ ﻪﻠﻤﺣ
Entering print_word_translate_arabic_fix ؟ﻩﺩﺍ

/usr/local/lib/python3.7/dist-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -64283
  warnings.warn("cmap value too big/small: %s" % cm)
/usr/local/lib/python3.7/dist-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -64284
  warnings.warn("cmap value too big/small: %s" % cm)
/usr/local/lib/python3.7/dist-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -64317
  warnings.warn("cmap value too big/small: %s" % cm)
/usr/local/lib/python3.7/dist-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -64330
  warnings.warn("cmap value too big/small: %s" % cm)
/usr/local/lib/python3.7/dist-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -64332
  warnings.warn("cmap value too big/small: %s" % cm)
/usr/local/lib/python3.7/dist-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -64334
  warnings.warn("cmap value too big/small: %s" % cm)
/usr/local/lib/python3.7/dist-packages/fpdf/tt

Entering create_pdf_instance() 
Entering rtl_arabic
Entering rtl_arabic
EXP= AME - Aide médicale d'État
COL= 1
HERE >>>DejaVuSans-Bold
Entering print_word_translate_arabic_fix ﯽﺘﻟﻭﺩ ﯽﮑﺷﺰﭘ ﯼﺎﻫ ﮏﻤﮐ - AME
ﯽﺘﻟﻭﺩ ﯽﮑﺷﺰﭘ ﯼﺎﻫ ﮏﻤﮐ - AME
length:25
cell width:46.66718518518517
pica:12
MOT 2 :  ﯽﺘﻟﻭﺩ ﯽﮑﺷﺰﭘ ﯼﺎﻫ ﮏﻤﮐ - AME 
MOT 2 (REV) : ﯽﺘﻟﻭﺩ ﯽﮑﺷﺰﭘ ﯼﺎﻫ ﮏﻤﮐ - AME 
Entering rtl_arabic
EXP= BCG - Vaccin bilié de Calmette et Guérin
COL= 1
HERE >>>DejaVuSans-Bold
Entering print_word_translate_arabic_fix Guérin ﻭ Calmette ﯼﻭﺍﺮﻔﺻ ﻦﺴﮐﺍﻭ - BCG
Guérin ﻭ Calmette ﯼﻭﺍﺮﻔﺻ ﻦﺴﮐﺍﻭ - BCG
length:36
cell width:46.66718518518517
pica:12
MOT 2 :  Guérin ﻭ Calmette ﯼﻭﺍﺮﻔﺻ ﻦﺴﮐﺍﻭ - BCG 
MOT 2 (REV) : Guérin ﻭ Calmette ﯼﻭﺍﺮﻔﺻ ﻦﺴﮐﺍﻭ - BCG 
Entering rtl_arabic
EXP= CSS (Complémentaire Santé Solidaire, ex-CMU)
COL= 1
HERE >>>DejaVuSans-Bold
Entering print_word_translate_arabic_fix (ﻖﺑﺎﺳ CMU ،Solidaire Santé Complementaire) CSS
(ﻖﺑﺎﺳ CMU ،Solidaire Santé Complementaire) CSS
length:46
cell width:46.66718518518517
pica:12
MOT 2 :

In [ ]:
import re
s = "CSS (کمپلینټیر سانټی سولیډیر، پخوانی CMU)"
s = "آیا ماشوم په دې وروستیو کې د یو چا سره اړیکه درلوده چې په ساري ناروغۍ اخته وي؟"
url_pattern = re.compile("[a-zA-Z]+")
vk = url_pattern.findall(s)
re.sub('[a-zA-Z]+', '£', s)


'£ (کمپلینټیر سانټی سولیډیر، پخوانی £)'

In [ ]:
s = "؟ﻱﻭ ﻪﺘﺧﺍ ۍﻍﻭﺭﺎﻧ ﻱﺭﺎﺳ ﻪﭘ ﯥﭼ ﻩﺩﻮﻟﺭﺩ ﻪﮑﯾړﺍ ﻩﴎ ﺎﭼ ﻮﯾ ﺩ ﯥﮐ ﻮﯿﺘﺳﻭﺭﻭ ﯤﺩ ﻪﭘ ﻡﻮﺷﺎﻣ ﺎﯾﺁ"
ori = "آیا ماشوم په دې وروستیو کې د یو چا سره اړیکه درلوده چې په ساري ناروغۍ اخته وي؟"
l = len(s)
w = 70 / 1.5
p = 12
vk = s.split(" ")
print (s + "\nlength:"+str(l)+ "\ncell width:"+str(w)+ "\npica:"+str(p))
z = ""
vz = []
for item in reversed(vk):
  print(item)
  if len(z) < w:
    z = z + item + " "
  else:
    vz.append(z.strip())
    z = ""
vz.append(z.strip())
vz.reverse()
print (vz)
vk = z.split(" ")
z =""
for item in reversed(vk):
  z = z + item + " "


print(z.strip())

؟ﻱﻭ ﻪﺘﺧﺍ ۍﻍﻭﺭﺎﻧ ﻱﺭﺎﺳ ﻪﭘ ﯥﭼ ﻩﺩﻮﻟﺭﺩ ﻪﮑﯾړﺍ ﻩﴎ ﺎﭼ ﻮﯾ ﺩ ﯥﮐ ﻮﯿﺘﺳﻭﺭﻭ ﯤﺩ ﻪﭘ ﻡﻮﺷﺎﻣ ﺎﯾﺁ
length:77
cell width:46.666666666666664
pica:12
ﺎﯾﺁ
ﻡﻮﺷﺎﻣ
ﻪﭘ
ﯤﺩ
ﻮﯿﺘﺳﻭﺭﻭ
ﯥﮐ
ﺩ
ﻮﯾ
ﺎﭼ
ﻩﴎ
ﻪﮑﯾړﺍ
ﻩﺩﻮﻟﺭﺩ
ﯥﭼ
ﻪﭘ
ﻱﺭﺎﺳ
ۍﻍﻭﺭﺎﻧ
ﻪﺘﺧﺍ
؟ﻱﻭ
['ﻪﭘ ﻱﺭﺎﺳ ۍﻍﻭﺭﺎﻧ ﻪﺘﺧﺍ ؟ﻱﻭ', 'ﺎﯾﺁ ﻡﻮﺷﺎﻣ ﻪﭘ ﯤﺩ ﻮﯿﺘﺳﻭﺭﻭ ﯥﮐ ﺩ ﻮﯾ ﺎﭼ ﻩﴎ ﻪﮑﯾړﺍ ﻩﺩﻮﻟﺭﺩ']
؟ﻱﻭ ﻪﺘﺧﺍ ۍﻍﻭﺭﺎﻧ ﻱﺭﺎﺳ ﻪﭘ


In [ ]:
vk_lang_dict

[{'alphabet': 'اآٱأإبتثجحخدذرزسشصضطظعغفقكلمنهةوؤيئىء',
  'credits': 'Sonia ZARROUK, Wafa TAHRI',
  'font-family': 'DejaVuSans',
  'font-family-bold': 'DejaVuSans-Bold',
  'font-path': '/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf',
  'glossary-subtitle': 'عربي فرنسي',
  'idx': 1,
  'language': 'Arabe',
  'meta': {'credits-fr': '',
   'credits-tr': '',
   'subtitle-fr': '',
   'subtitle-tr': '',
   'title-fr': '',
   'title-tr': ''},
  'text-direction': 'rtl',
  'trigramme': 'ams',
  'unicode_substition': [],
  'update': 'false',
  'uri': '134rEeVux-FvSxPt4EO4OM1M8PnGHa6P1ewtmnsnRfrs'},
 {'alphabet': 'AaBbCcDdEeFfGgHhIiJjKkLlMmNnOoPpQqRrSsTtUuVvWwXxYyZz',
  'credits': 'Khalida BENHEDDER',
  'font-family': 'NotoSerif-Regular',
  'font-family-bold': 'NotoSerif-Bold',
  'glossary-subtitle': 'English-French',
  'idx': 2,
  'language': 'Anglais',
  'meta': {'credits-fr': '',
   'credits-tr': '',
   'subtitle-fr': '',
   'subtitle-tr': '',
   'title-fr': '',
   'title-tr': ''},
  'text-dir

In [ ]:
re.sub(url_pattern, lambda m:m.group(0)[::-1], s)

'SSC (کمپلینټیر سانټی سولیډیر، پخوانی UMC)'

In [ ]:

import re
url_pattern = re.compile("[a-zA-Zéèêëàùô0-9\.]+")
s = "C.S.S (کمپلینټیر سانټی سولیډیر، پخوانی CMU)"
s = re.sub(url_pattern, lambda m:m.group(0)[::-1], s) 

arabic_string = reshaper.reshape(s)
arabic_string = arabic_string[::-1]
# problème d'inversion des parenthèses
arabic_string = arabic_string.replace('(', '§')
arabic_string = arabic_string.replace(')', '(')    
arabic_string = arabic_string.replace('§', ')')

print(arabic_string)

(CMU ﯽﻧﺍﻮﺨﭘ ،ﺮﯾډﯽﻟﻮﺳ ﯼټﻥﺎﺳ ﺮﯾټﻦﯿﻠﭙﻤﮐ) C.S.S


In [ ]:
# sauvegarde du fichier de paramétrage
'''
with open('med_vac_synthese.json', 'w') as fp:
    json.dump(vk_lang_dict, fp)
'''

"\nwith open('med_vac_synthese.json', 'w') as fp:\n    json.dump(vk_lang_dict, fp)\n"

In [ ]:
from fpdf import FPDF


from arabic_reshaper import ArabicReshaper
configuration = {
    'language': "Farsi",
    'delete_harakat': False,
    'support_ligatures': True,
    'RIAL SIGN': True,  # Replace ر ي ا ل with ﷼
}
reshaper = ArabicReshaper(configuration=configuration)


pdf2 = FPDF()

# compression is not yet supported in py3k version
pdf2.compress = False
pdf2.add_page()
# Unicode is not yet supported in the py3k version; use windows-1252 standard font
pdf2.add_font('NotoSansArabic-Regular','','/usr/share/fonts/truetype/noto/NotoSansArabic-Regular.ttf', uni=True)  
pdf2.add_font('NotoNaskhArabic-Regular','','/usr/share/fonts/truetype/noto/NotoNaskhArabic-Regular.ttf', uni=True)  

pdf2.set_font('NotoNaskhArabic-Regular', '', 14)  
pdf2.ln(10)
s = rtl_arabic("د کورنۍ په بل غړي باندې د بخار برید؟", pdf2)
s = "؟ﺪﯾﺮﺑ ﺭﺎﺨﺑ ﺩ ﯤﺪﻧﺎﺑ ﻱړﻍ ﻞﺑ ﻪﭘ ۍﻥﺭﻮﮐ ﺩ"
s = reshaper.reshape(s)
pdf2.write(5, s)
pdf2.output('py3k.pdf', 'F')



/usr/local/lib/python3.7/dist-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -64332
  warnings.warn("cmap value too big/small: %s" % cm)
/usr/local/lib/python3.7/dist-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -64387
  warnings.warn("cmap value too big/small: %s" % cm)
/usr/local/lib/python3.7/dist-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -64473
  warnings.warn("cmap value too big/small: %s" % cm)
/usr/local/lib/python3.7/dist-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -64496
  warnings.warn("cmap value too big/small: %s" % cm)
/usr/local/lib/python3.7/dist-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -65151
  warnings.warn("cmap value too big/small: %s" % cm)
/usr/local/lib/python3.7/dist-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -65153
  warnings.warn("cmap value too big/small: %s" % cm)
/usr/local/lib/python3.7/dist-packages/fpdf/tt

''

In [ ]:
pdf = FPDF('P', 'mm', 'A4')
im_width = 4001
im_height = 7001
page_width = 210
page_height = 297
left_margin = 0
right_margin = 0
album = []
album.append("26363.jpg")
for image in album:
  pdf.add_page()
  pdf.image(image, left_margin, right_margin, page_width, page_height)
pdf.output("/content/drive/MyDrive/Trad-Union/Corpus/ASAMLA/cover.pdf", "F")

''

In [ ]:
#https://stackoverflow.com/questions/27327513/create-pdf-from-a-list-of-images
from fpdf import FPDF
import os, sys, glob
from tqdm import tqdm

pdf = FPDF('P', 'mm', 'A4')
im_width = 1654
im_height = 2339

aspect_ratio = im_height/im_width
page_width = 210
# page_height = aspect_ratio * page_width
page_height = 297
left_margin = 0
right_margin = 0


dossier = "ams"
sous_dossier = "lex-ams-fr"
os.chdir("/content/drive/MyDrive/Trad-Union/Corpus/ASAMLA/" +  dossier + "/" + sous_dossier) 
album = []

for i in range(18, 0, -1):
  fn = "med-vac-" + sous_dossier + "-_" + str(i) + ".jpg"
  album.append(fn)

# imagelist is the list with all image filenames
for image in album:
  pdf.add_page()
  pdf.image(image, left_margin, right_margin, page_width, page_height)
pdf.output("/content/drive/MyDrive/Trad-Union/Corpus/ASAMLA/med-vac-ams-fr-v2.1-RTL.pdf", "F")
print('Conversion completed!')


Conversion completed!


In [ ]:
vk_lang_dict

[{'alphabet': '',
  'credits': 'Massouda HEMATYAR',
  'font-family': 'Adobe Arabic Regular',
  'font-family-bold': 'DejaVuSans-Bold',
  'glossary-subtitle': '',
  'idx': 13,
  'language': 'Pashto',
  'meta': {'credits-fr': '',
   'credits-tr': '',
   'subtitle-fr': '',
   'subtitle-tr': '',
   'title-fr': '',
   'title-tr': ''},
  'text-direction': 'rtl',
  'trigramme': 'pst',
  'unicode_substition': [],
  'update': 'false',
  'uri': '1IDUQ_SlSOMWX87NrwwZ94SBcNYEoPx-hz7lgC5c5FO8'}]